In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows',98)
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [19]:
df=pd.read_csv("./data/MiceImputed_weather.csv")
df=df.iloc[:,1:]
df

,location,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindGustDir,Sunshine,Evaporation,TodayRain,...,Cloud9am,Humidity9am,WindSpeed9am,Temp9am,Pressure3pm,WindDir3pm,Cloud3pm,Humidity3pm,WindSpeed3pm,Temp3pm
0,45.0,0.0,12.6,23.2,0.5,6.4,7.0,17.627755,4.696022,0.0,...,0.0,66.0,1.6,18.3,998.7,9.0,6.0,51.0,4.2,19.3
1,45.0,1.0,10.6,24.1,0.0,6.5,15.0,21.060788,6.238979,0.0,...,3.0,30.0,5.1,18.6,1000.8,0.0,2.0,35.0,2.7,20.5
2,45.0,2.0,14.2,25.1,0.0,7.0,7.0,24.272884,6.987526,0.0,...,0.0,26.0,3.1,22.2,1002.8,0.0,0.0,31.0,2.1,22.9
3,45.0,3.0,12.1,21.1,0.0,6.1,7.0,14.398951,3.787547,0.0,...,9.0,66.0,0.8,17.0,1005.5,2.0,3.0,70.0,2.8,18.3
4,45.0,4.0,11.9,15.4,0.0,6.5,7.0,7.348844,1.284132,0.0,...,8.0,90.0,3.8,14.7,1013.2,6.0,9.0,89.0,4.6,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343701,17.0,3647.0,9.8,20.3,0.0,4.8,14.0,19.648829,5.552254,0.0,...,4.0,48.0,1.2,14.4,1006.8,8.0,1.0,35.0,2.4,19.5
343702,17.0,3648.0,7.9,21.5,0.0,5.4,14.0,18.247347,5.144071,0.0,...,0.0,50.0,1.7,15.9,1006.5,15.0,7.0,42.0,2.2,20.6
343703,17.0,3649.0,8.3,19.9,0.0,3.6,11.0,20.493692,5.126407,0.0,...,2.0,66.0,1.2,13.0,1014.3,8.0,0.0,37.0,2.7,18.9
343704,17.0,3650.0,8.2,22.7,0.0,4.1,11.0,22.913542,5.828915,0.0,...,0.0,59.0,1.0,15.5,1014.0,11.0,0.0,39.0,2.6,21.3


In [20]:
Y=df.TomorrowRain
X=df.drop("TomorrowRain",axis=1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify = Y, random_state = 0)

print(X_train.shape ,X_val.shape, X_test.shape)

(274964, 22) (54993, 22) (68742, 22)


In [23]:
X_train.columns

Index(['location', 'Date', 'MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed',
       'WindGustDir', 'Sunshine', 'Evaporation', 'TodayRain', 'Pressure9am',
       'WindDir9am', 'Cloud9am', 'Humidity9am', 'WindSpeed9am', 'Temp9am',
       'Pressure3pm', 'WindDir3pm', 'Cloud3pm', 'Humidity3pm', 'WindSpeed3pm',
       'Temp3pm'],
      dtype='object')

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    #roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [24]:
import numpy as np

from catboost import CatBoostClassifier, Pool

# initialize data


test_data = catboost_pool = Pool(X_val, 
                                 y_val)

model = CatBoostClassifier(iterations=2,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class)
print("proba = ", preds_proba)

0:	learn: 0.4669291	total: 42.1ms	remaining: 42.1ms
1:	learn: 0.4598776	total: 54.7ms	remaining: 0us
class =  [0. 0. 0. ... 0. 0. 0.]
proba =  [[0.60727766 0.39272234]
 [0.71903812 0.28096188]
 [0.91290318 0.08709682]
 ...
 [0.79337035 0.20662965]
 [0.91290318 0.08709682]
 [0.57536118 0.42463882]]


In [25]:
get_clf_eval(y_test, preds_class, preds_proba)

오차 행렬
[[53908   794]
 [12614  1426]]
정확도: 0.8050, 정밀도: 0.6423, 재현율: 0.1016,    F1: 0.1754


In [26]:
model = CatBoostClassifier(iterations=10,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print("class = ", preds_class)
print("proba = ", preds_proba)

0:	learn: 0.4669291	total: 45.6ms	remaining: 410ms
1:	learn: 0.4598776	total: 58.6ms	remaining: 235ms
2:	learn: 0.4555731	total: 71.9ms	remaining: 168ms
3:	learn: 0.4535603	total: 85ms	remaining: 127ms
4:	learn: 0.4510265	total: 98.3ms	remaining: 98.3ms
5:	learn: 0.4494675	total: 111ms	remaining: 74.1ms
6:	learn: 0.4470158	total: 125ms	remaining: 53.7ms
7:	learn: 0.4450430	total: 138ms	remaining: 34.4ms
8:	learn: 0.4433702	total: 151ms	remaining: 16.8ms
9:	learn: 0.4410425	total: 165ms	remaining: 0us
class =  [0. 0. 0. ... 0. 0. 0.]
proba =  [[0.72657379 0.27342621]
 [0.7203475  0.2796525 ]
 [0.94205839 0.05794161]
 ...
 [0.80859631 0.19140369]
 [0.92572059 0.07427941]
 [0.58381754 0.41618246]]


In [27]:
get_clf_eval(y_test, preds_class, preds_proba)

오차 행렬
[[52881  1821]
 [11286  2754]]
정확도: 0.8093, 정밀도: 0.6020, 재현율: 0.1962,    F1: 0.2959


In [28]:
model = CatBoostClassifier(iterations=100,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4669291	total: 52.5ms	remaining: 5.2s
1:	learn: 0.4598776	total: 65.7ms	remaining: 3.22s
2:	learn: 0.4555731	total: 80.1ms	remaining: 2.59s
3:	learn: 0.4535603	total: 93.5ms	remaining: 2.24s
4:	learn: 0.4510265	total: 107ms	remaining: 2.02s
5:	learn: 0.4494675	total: 119ms	remaining: 1.86s
6:	learn: 0.4470158	total: 133ms	remaining: 1.76s
7:	learn: 0.4450430	total: 146ms	remaining: 1.68s
8:	learn: 0.4433702	total: 159ms	remaining: 1.61s
9:	learn: 0.4410425	total: 173ms	remaining: 1.56s
10:	learn: 0.4399006	total: 186ms	remaining: 1.51s
11:	learn: 0.4391415	total: 202ms	remaining: 1.48s
12:	learn: 0.4384775	total: 213ms	remaining: 1.43s
13:	learn: 0.4377789	total: 225ms	remaining: 1.38s
14:	learn: 0.4367890	total: 239ms	remaining: 1.35s
15:	learn: 0.4353891	total: 254ms	remaining: 1.33s
16:	learn: 0.4342300	total: 267ms	remaining: 1.3s
17:	learn: 0.4330128	total: 280ms	remaining: 1.27s
18:	learn: 0.4321831	total: 294ms	remaining: 1.25s
19:	learn: 0.4314021	total: 307ms	remai

In [15]:
model = CatBoostClassifier(iterations=200,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4671737	total: 56.3ms	remaining: 11.2s
1:	learn: 0.4593466	total: 67.6ms	remaining: 6.69s
2:	learn: 0.4560656	total: 77.8ms	remaining: 5.11s
3:	learn: 0.4538639	total: 87.5ms	remaining: 4.29s
4:	learn: 0.4522100	total: 98.7ms	remaining: 3.85s
5:	learn: 0.4500863	total: 109ms	remaining: 3.52s
6:	learn: 0.4479436	total: 119ms	remaining: 3.28s
7:	learn: 0.4458394	total: 129ms	remaining: 3.09s
8:	learn: 0.4445369	total: 139ms	remaining: 2.95s
9:	learn: 0.4437036	total: 149ms	remaining: 2.83s
10:	learn: 0.4423625	total: 159ms	remaining: 2.73s
11:	learn: 0.4416281	total: 168ms	remaining: 2.64s
12:	learn: 0.4406580	total: 178ms	remaining: 2.56s
13:	learn: 0.4392308	total: 187ms	remaining: 2.49s
14:	learn: 0.4378992	total: 197ms	remaining: 2.43s
15:	learn: 0.4368076	total: 207ms	remaining: 2.38s
16:	learn: 0.4354127	total: 218ms	remaining: 2.35s
17:	learn: 0.4338663	total: 228ms	remaining: 2.31s
18:	learn: 0.4332205	total: 238ms	remaining: 2.27s
19:	learn: 0.4323554	total: 249ms	re

172:	learn: 0.3804089	total: 1.9s	remaining: 296ms
173:	learn: 0.3803181	total: 1.91s	remaining: 285ms
174:	learn: 0.3802246	total: 1.92s	remaining: 274ms
175:	learn: 0.3801391	total: 1.93s	remaining: 263ms
176:	learn: 0.3800368	total: 1.94s	remaining: 252ms
177:	learn: 0.3799416	total: 1.95s	remaining: 241ms
178:	learn: 0.3797738	total: 1.96s	remaining: 230ms
179:	learn: 0.3795041	total: 1.97s	remaining: 219ms
180:	learn: 0.3793354	total: 1.99s	remaining: 208ms
181:	learn: 0.3792103	total: 2s	remaining: 197ms
182:	learn: 0.3790062	total: 2s	remaining: 186ms
183:	learn: 0.3787200	total: 2.02s	remaining: 175ms
184:	learn: 0.3785200	total: 2.03s	remaining: 164ms
185:	learn: 0.3783291	total: 2.04s	remaining: 153ms
186:	learn: 0.3780935	total: 2.05s	remaining: 142ms
187:	learn: 0.3779595	total: 2.06s	remaining: 132ms
188:	learn: 0.3777992	total: 2.07s	remaining: 121ms
189:	learn: 0.3776800	total: 2.08s	remaining: 110ms
190:	learn: 0.3774339	total: 2.09s	remaining: 98.7ms
191:	learn: 0.3772

In [30]:
model = CatBoostClassifier(iterations=500,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4669291	total: 48.9ms	remaining: 24.4s
1:	learn: 0.4598776	total: 61.5ms	remaining: 15.3s
2:	learn: 0.4555731	total: 75.1ms	remaining: 12.4s
3:	learn: 0.4535603	total: 88.4ms	remaining: 11s
4:	learn: 0.4510265	total: 101ms	remaining: 10s
5:	learn: 0.4494675	total: 113ms	remaining: 9.34s
6:	learn: 0.4470158	total: 127ms	remaining: 8.96s
7:	learn: 0.4450430	total: 140ms	remaining: 8.62s
8:	learn: 0.4433702	total: 153ms	remaining: 8.35s
9:	learn: 0.4410425	total: 168ms	remaining: 8.21s
10:	learn: 0.4399006	total: 180ms	remaining: 8.02s
11:	learn: 0.4391415	total: 193ms	remaining: 7.86s
12:	learn: 0.4384775	total: 205ms	remaining: 7.66s
13:	learn: 0.4377789	total: 216ms	remaining: 7.51s
14:	learn: 0.4367890	total: 229ms	remaining: 7.4s
15:	learn: 0.4353891	total: 243ms	remaining: 7.36s
16:	learn: 0.4342300	total: 257ms	remaining: 7.3s
17:	learn: 0.4330128	total: 269ms	remaining: 7.21s
18:	learn: 0.4321831	total: 283ms	remaining: 7.17s
19:	learn: 0.4314021	total: 296ms	remaining

164:	learn: 0.3818528	total: 2.27s	remaining: 4.62s
165:	learn: 0.3816500	total: 2.29s	remaining: 4.6s
166:	learn: 0.3815035	total: 2.3s	remaining: 4.59s
167:	learn: 0.3812845	total: 2.32s	remaining: 4.58s
168:	learn: 0.3809485	total: 2.33s	remaining: 4.56s
169:	learn: 0.3807918	total: 2.34s	remaining: 4.55s
170:	learn: 0.3806188	total: 2.35s	remaining: 4.53s
171:	learn: 0.3804925	total: 2.37s	remaining: 4.51s
172:	learn: 0.3804084	total: 2.38s	remaining: 4.51s
173:	learn: 0.3803410	total: 2.4s	remaining: 4.49s
174:	learn: 0.3802788	total: 2.41s	remaining: 4.47s
175:	learn: 0.3801767	total: 2.42s	remaining: 4.46s
176:	learn: 0.3799736	total: 2.43s	remaining: 4.44s
177:	learn: 0.3797986	total: 2.45s	remaining: 4.43s
178:	learn: 0.3797118	total: 2.46s	remaining: 4.41s
179:	learn: 0.3796187	total: 2.47s	remaining: 4.4s
180:	learn: 0.3794577	total: 2.49s	remaining: 4.38s
181:	learn: 0.3792760	total: 2.5s	remaining: 4.37s
182:	learn: 0.3791326	total: 2.51s	remaining: 4.35s
183:	learn: 0.378

325:	learn: 0.3605143	total: 4.46s	remaining: 2.38s
326:	learn: 0.3604149	total: 4.47s	remaining: 2.37s
327:	learn: 0.3603327	total: 4.49s	remaining: 2.35s
328:	learn: 0.3602620	total: 4.5s	remaining: 2.34s
329:	learn: 0.3602074	total: 4.52s	remaining: 2.33s
330:	learn: 0.3601275	total: 4.53s	remaining: 2.31s
331:	learn: 0.3600212	total: 4.54s	remaining: 2.3s
332:	learn: 0.3599543	total: 4.55s	remaining: 2.28s
333:	learn: 0.3599124	total: 4.57s	remaining: 2.27s
334:	learn: 0.3597028	total: 4.58s	remaining: 2.26s
335:	learn: 0.3596030	total: 4.59s	remaining: 2.24s
336:	learn: 0.3593946	total: 4.61s	remaining: 2.23s
337:	learn: 0.3592954	total: 4.62s	remaining: 2.21s
338:	learn: 0.3592067	total: 4.63s	remaining: 2.2s
339:	learn: 0.3591232	total: 4.65s	remaining: 2.19s
340:	learn: 0.3590130	total: 4.66s	remaining: 2.17s
341:	learn: 0.3589291	total: 4.67s	remaining: 2.16s
342:	learn: 0.3588623	total: 4.69s	remaining: 2.15s
343:	learn: 0.3588205	total: 4.7s	remaining: 2.13s
344:	learn: 0.35

491:	learn: 0.3474931	total: 6.65s	remaining: 108ms
492:	learn: 0.3474684	total: 6.66s	remaining: 94.6ms
493:	learn: 0.3473978	total: 6.68s	remaining: 81.1ms
494:	learn: 0.3473086	total: 6.69s	remaining: 67.6ms
495:	learn: 0.3472626	total: 6.7s	remaining: 54ms
496:	learn: 0.3472227	total: 6.71s	remaining: 40.5ms
497:	learn: 0.3471645	total: 6.73s	remaining: 27ms
498:	learn: 0.3471201	total: 6.74s	remaining: 13.5ms
499:	learn: 0.3470556	total: 6.75s	remaining: 0us
오차 행렬
[[52366  2336]
 [ 8073  5967]]
정확도: 0.8486, 정밀도: 0.7187, 재현율: 0.4250,    F1: 0.5341


In [31]:
model = CatBoostClassifier(iterations=1000,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4669291	total: 54.5ms	remaining: 54.4s
1:	learn: 0.4598776	total: 67.8ms	remaining: 33.8s
2:	learn: 0.4555731	total: 82.2ms	remaining: 27.3s
3:	learn: 0.4535603	total: 95.5ms	remaining: 23.8s
4:	learn: 0.4510265	total: 109ms	remaining: 21.6s
5:	learn: 0.4494675	total: 122ms	remaining: 20.2s
6:	learn: 0.4470158	total: 137ms	remaining: 19.4s
7:	learn: 0.4450430	total: 150ms	remaining: 18.6s
8:	learn: 0.4433702	total: 163ms	remaining: 18s
9:	learn: 0.4410425	total: 178ms	remaining: 17.6s
10:	learn: 0.4399006	total: 191ms	remaining: 17.2s
11:	learn: 0.4391415	total: 205ms	remaining: 16.9s
12:	learn: 0.4384775	total: 217ms	remaining: 16.5s
13:	learn: 0.4377789	total: 229ms	remaining: 16.2s
14:	learn: 0.4367890	total: 242ms	remaining: 15.9s
15:	learn: 0.4353891	total: 257ms	remaining: 15.8s
16:	learn: 0.4342300	total: 272ms	remaining: 15.7s
17:	learn: 0.4330128	total: 284ms	remaining: 15.5s
18:	learn: 0.4321831	total: 299ms	remaining: 15.4s
19:	learn: 0.4314021	total: 314ms	remai

173:	learn: 0.3803410	total: 2.43s	remaining: 11.6s
174:	learn: 0.3802788	total: 2.44s	remaining: 11.5s
175:	learn: 0.3801767	total: 2.46s	remaining: 11.5s
176:	learn: 0.3799736	total: 2.47s	remaining: 11.5s
177:	learn: 0.3797986	total: 2.48s	remaining: 11.5s
178:	learn: 0.3797118	total: 2.5s	remaining: 11.4s
179:	learn: 0.3796187	total: 2.51s	remaining: 11.4s
180:	learn: 0.3794577	total: 2.52s	remaining: 11.4s
181:	learn: 0.3792760	total: 2.54s	remaining: 11.4s
182:	learn: 0.3791326	total: 2.55s	remaining: 11.4s
183:	learn: 0.3789641	total: 2.57s	remaining: 11.4s
184:	learn: 0.3787762	total: 2.58s	remaining: 11.4s
185:	learn: 0.3785949	total: 2.59s	remaining: 11.4s
186:	learn: 0.3784747	total: 2.61s	remaining: 11.3s
187:	learn: 0.3783812	total: 2.62s	remaining: 11.3s
188:	learn: 0.3781390	total: 2.63s	remaining: 11.3s
189:	learn: 0.3780082	total: 2.65s	remaining: 11.3s
190:	learn: 0.3778654	total: 2.66s	remaining: 11.3s
191:	learn: 0.3777400	total: 2.67s	remaining: 11.3s
192:	learn: 0

334:	learn: 0.3597028	total: 4.63s	remaining: 9.2s
335:	learn: 0.3596030	total: 4.65s	remaining: 9.19s
336:	learn: 0.3593946	total: 4.67s	remaining: 9.18s
337:	learn: 0.3592954	total: 4.68s	remaining: 9.17s
338:	learn: 0.3592067	total: 4.7s	remaining: 9.15s
339:	learn: 0.3591232	total: 4.71s	remaining: 9.14s
340:	learn: 0.3590130	total: 4.72s	remaining: 9.12s
341:	learn: 0.3589291	total: 4.73s	remaining: 9.11s
342:	learn: 0.3588623	total: 4.75s	remaining: 9.09s
343:	learn: 0.3588205	total: 4.76s	remaining: 9.07s
344:	learn: 0.3587333	total: 4.77s	remaining: 9.06s
345:	learn: 0.3585987	total: 4.78s	remaining: 9.04s
346:	learn: 0.3585256	total: 4.8s	remaining: 9.03s
347:	learn: 0.3583768	total: 4.81s	remaining: 9.02s
348:	learn: 0.3582779	total: 4.83s	remaining: 9.01s
349:	learn: 0.3582028	total: 4.84s	remaining: 8.99s
350:	learn: 0.3581259	total: 4.86s	remaining: 8.98s
351:	learn: 0.3580492	total: 4.87s	remaining: 8.96s
352:	learn: 0.3578975	total: 4.88s	remaining: 8.95s
353:	learn: 0.3

505:	learn: 0.3466243	total: 6.99s	remaining: 6.82s
506:	learn: 0.3465455	total: 7s	remaining: 6.81s
507:	learn: 0.3465047	total: 7.01s	remaining: 6.79s
508:	learn: 0.3464648	total: 7.03s	remaining: 6.78s
509:	learn: 0.3463993	total: 7.04s	remaining: 6.76s
510:	learn: 0.3463668	total: 7.05s	remaining: 6.75s
511:	learn: 0.3463263	total: 7.07s	remaining: 6.74s
512:	learn: 0.3462337	total: 7.08s	remaining: 6.72s
513:	learn: 0.3461588	total: 7.09s	remaining: 6.71s
514:	learn: 0.3461233	total: 7.11s	remaining: 6.7s
515:	learn: 0.3460719	total: 7.12s	remaining: 6.68s
516:	learn: 0.3460317	total: 7.13s	remaining: 6.67s
517:	learn: 0.3458984	total: 7.15s	remaining: 6.65s
518:	learn: 0.3458184	total: 7.16s	remaining: 6.64s
519:	learn: 0.3457705	total: 7.18s	remaining: 6.63s
520:	learn: 0.3457208	total: 7.19s	remaining: 6.61s
521:	learn: 0.3456518	total: 7.21s	remaining: 6.6s
522:	learn: 0.3455819	total: 7.22s	remaining: 6.58s
523:	learn: 0.3455278	total: 7.23s	remaining: 6.57s
524:	learn: 0.345

664:	learn: 0.3379290	total: 9.18s	remaining: 4.62s
665:	learn: 0.3378859	total: 9.19s	remaining: 4.61s
666:	learn: 0.3378427	total: 9.21s	remaining: 4.6s
667:	learn: 0.3377709	total: 9.22s	remaining: 4.58s
668:	learn: 0.3377024	total: 9.23s	remaining: 4.57s
669:	learn: 0.3376612	total: 9.25s	remaining: 4.56s
670:	learn: 0.3376297	total: 9.26s	remaining: 4.54s
671:	learn: 0.3376018	total: 9.28s	remaining: 4.53s
672:	learn: 0.3375460	total: 9.29s	remaining: 4.51s
673:	learn: 0.3375163	total: 9.3s	remaining: 4.5s
674:	learn: 0.3374596	total: 9.32s	remaining: 4.49s
675:	learn: 0.3374071	total: 9.33s	remaining: 4.47s
676:	learn: 0.3373753	total: 9.36s	remaining: 4.47s
677:	learn: 0.3373237	total: 9.37s	remaining: 4.45s
678:	learn: 0.3372496	total: 9.39s	remaining: 4.44s
679:	learn: 0.3370272	total: 9.4s	remaining: 4.42s
680:	learn: 0.3369979	total: 9.42s	remaining: 4.41s
681:	learn: 0.3369569	total: 9.43s	remaining: 4.4s
682:	learn: 0.3368967	total: 9.44s	remaining: 4.38s
683:	learn: 0.336

828:	learn: 0.3303935	total: 11.6s	remaining: 2.39s
829:	learn: 0.3303537	total: 11.6s	remaining: 2.37s
830:	learn: 0.3303339	total: 11.6s	remaining: 2.36s
831:	learn: 0.3302952	total: 11.6s	remaining: 2.35s
832:	learn: 0.3302793	total: 11.6s	remaining: 2.33s
833:	learn: 0.3302511	total: 11.6s	remaining: 2.32s
834:	learn: 0.3302166	total: 11.7s	remaining: 2.3s
835:	learn: 0.3301788	total: 11.7s	remaining: 2.29s
836:	learn: 0.3301476	total: 11.7s	remaining: 2.27s
837:	learn: 0.3301198	total: 11.7s	remaining: 2.26s
838:	learn: 0.3300806	total: 11.7s	remaining: 2.25s
839:	learn: 0.3300486	total: 11.7s	remaining: 2.23s
840:	learn: 0.3300074	total: 11.7s	remaining: 2.22s
841:	learn: 0.3299769	total: 11.8s	remaining: 2.21s
842:	learn: 0.3299438	total: 11.8s	remaining: 2.19s
843:	learn: 0.3299014	total: 11.8s	remaining: 2.18s
844:	learn: 0.3298687	total: 11.8s	remaining: 2.16s
845:	learn: 0.3298198	total: 11.8s	remaining: 2.15s
846:	learn: 0.3297889	total: 11.8s	remaining: 2.13s
847:	learn: 0

995:	learn: 0.3243595	total: 13.9s	remaining: 56ms
996:	learn: 0.3243084	total: 14s	remaining: 42ms
997:	learn: 0.3242815	total: 14s	remaining: 28ms
998:	learn: 0.3242443	total: 14s	remaining: 14ms
999:	learn: 0.3242173	total: 14s	remaining: 0us
오차 행렬
[[52373  2329]
 [ 7411  6629]]
정확도: 0.8583, 정밀도: 0.7400, 재현율: 0.4722,    F1: 0.5765


In [36]:
model = CatBoostClassifier(iterations=1000,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4548989	total: 42.3ms	remaining: 42.2s
1:	learn: 0.4480689	total: 63.1ms	remaining: 31.5s
2:	learn: 0.4434459	total: 83.7ms	remaining: 27.8s
3:	learn: 0.4386681	total: 103ms	remaining: 25.5s
4:	learn: 0.4345743	total: 124ms	remaining: 24.8s
5:	learn: 0.4307806	total: 149ms	remaining: 24.6s
6:	learn: 0.4282029	total: 169ms	remaining: 24s
7:	learn: 0.4264150	total: 190ms	remaining: 23.5s
8:	learn: 0.4241899	total: 210ms	remaining: 23.1s
9:	learn: 0.4217375	total: 230ms	remaining: 22.7s
10:	learn: 0.4204619	total: 253ms	remaining: 22.7s
11:	learn: 0.4190606	total: 274ms	remaining: 22.5s
12:	learn: 0.4172735	total: 296ms	remaining: 22.5s
13:	learn: 0.4151833	total: 316ms	remaining: 22.2s
14:	learn: 0.4139492	total: 336ms	remaining: 22.1s
15:	learn: 0.4123291	total: 357ms	remaining: 21.9s
16:	learn: 0.4112445	total: 378ms	remaining: 21.8s
17:	learn: 0.4095297	total: 400ms	remaining: 21.8s
18:	learn: 0.4082762	total: 421ms	remaining: 21.7s
19:	learn: 0.4067103	total: 442ms	remain

162:	learn: 0.3197208	total: 3.67s	remaining: 18.9s
163:	learn: 0.3193866	total: 3.7s	remaining: 18.9s
164:	learn: 0.3190941	total: 3.72s	remaining: 18.8s
165:	learn: 0.3187404	total: 3.74s	remaining: 18.8s
166:	learn: 0.3185212	total: 3.76s	remaining: 18.8s
167:	learn: 0.3182247	total: 3.79s	remaining: 18.7s
168:	learn: 0.3180120	total: 3.81s	remaining: 18.7s
169:	learn: 0.3175018	total: 3.83s	remaining: 18.7s
170:	learn: 0.3170472	total: 3.85s	remaining: 18.7s
171:	learn: 0.3168034	total: 3.88s	remaining: 18.7s
172:	learn: 0.3161359	total: 3.9s	remaining: 18.7s
173:	learn: 0.3158492	total: 3.92s	remaining: 18.6s
174:	learn: 0.3155228	total: 3.95s	remaining: 18.6s
175:	learn: 0.3153431	total: 3.97s	remaining: 18.6s
176:	learn: 0.3151219	total: 3.99s	remaining: 18.5s
177:	learn: 0.3148524	total: 4.01s	remaining: 18.5s
178:	learn: 0.3145872	total: 4.03s	remaining: 18.5s
179:	learn: 0.3142515	total: 4.05s	remaining: 18.5s
180:	learn: 0.3138915	total: 4.07s	remaining: 18.4s
181:	learn: 0.

328:	learn: 0.2782023	total: 7.49s	remaining: 15.3s
329:	learn: 0.2779704	total: 7.52s	remaining: 15.3s
330:	learn: 0.2777476	total: 7.54s	remaining: 15.2s
331:	learn: 0.2775502	total: 7.57s	remaining: 15.2s
332:	learn: 0.2773843	total: 7.59s	remaining: 15.2s
333:	learn: 0.2771513	total: 7.61s	remaining: 15.2s
334:	learn: 0.2769501	total: 7.63s	remaining: 15.2s
335:	learn: 0.2767241	total: 7.66s	remaining: 15.1s
336:	learn: 0.2765176	total: 7.68s	remaining: 15.1s
337:	learn: 0.2763825	total: 7.7s	remaining: 15.1s
338:	learn: 0.2762098	total: 7.72s	remaining: 15.1s
339:	learn: 0.2760348	total: 7.74s	remaining: 15s
340:	learn: 0.2757625	total: 7.76s	remaining: 15s
341:	learn: 0.2755763	total: 7.79s	remaining: 15s
342:	learn: 0.2753953	total: 7.81s	remaining: 15s
343:	learn: 0.2752276	total: 7.83s	remaining: 14.9s
344:	learn: 0.2750925	total: 7.85s	remaining: 14.9s
345:	learn: 0.2748510	total: 7.87s	remaining: 14.9s
346:	learn: 0.2746923	total: 7.89s	remaining: 14.9s
347:	learn: 0.2744825

494:	learn: 0.2503111	total: 11.3s	remaining: 11.5s
495:	learn: 0.2501377	total: 11.3s	remaining: 11.5s
496:	learn: 0.2500618	total: 11.3s	remaining: 11.5s
497:	learn: 0.2499000	total: 11.4s	remaining: 11.5s
498:	learn: 0.2497760	total: 11.4s	remaining: 11.4s
499:	learn: 0.2494864	total: 11.4s	remaining: 11.4s
500:	learn: 0.2493035	total: 11.4s	remaining: 11.4s
501:	learn: 0.2491594	total: 11.5s	remaining: 11.4s
502:	learn: 0.2490186	total: 11.5s	remaining: 11.4s
503:	learn: 0.2488668	total: 11.5s	remaining: 11.3s
504:	learn: 0.2487275	total: 11.5s	remaining: 11.3s
505:	learn: 0.2485226	total: 11.6s	remaining: 11.3s
506:	learn: 0.2483541	total: 11.6s	remaining: 11.3s
507:	learn: 0.2481164	total: 11.6s	remaining: 11.2s
508:	learn: 0.2479589	total: 11.6s	remaining: 11.2s
509:	learn: 0.2477158	total: 11.7s	remaining: 11.2s
510:	learn: 0.2475544	total: 11.7s	remaining: 11.2s
511:	learn: 0.2474126	total: 11.7s	remaining: 11.2s
512:	learn: 0.2471953	total: 11.7s	remaining: 11.1s
513:	learn: 

656:	learn: 0.2279101	total: 15.1s	remaining: 7.86s
657:	learn: 0.2278456	total: 15.1s	remaining: 7.84s
658:	learn: 0.2276956	total: 15.1s	remaining: 7.81s
659:	learn: 0.2275725	total: 15.1s	remaining: 7.79s
660:	learn: 0.2273916	total: 15.1s	remaining: 7.77s
661:	learn: 0.2273207	total: 15.2s	remaining: 7.75s
662:	learn: 0.2271959	total: 15.2s	remaining: 7.72s
663:	learn: 0.2270585	total: 15.2s	remaining: 7.7s
664:	learn: 0.2269298	total: 15.2s	remaining: 7.68s
665:	learn: 0.2268430	total: 15.3s	remaining: 7.65s
666:	learn: 0.2267536	total: 15.3s	remaining: 7.63s
667:	learn: 0.2266649	total: 15.3s	remaining: 7.61s
668:	learn: 0.2265309	total: 15.3s	remaining: 7.58s
669:	learn: 0.2264624	total: 15.4s	remaining: 7.56s
670:	learn: 0.2262954	total: 15.4s	remaining: 7.54s
671:	learn: 0.2261747	total: 15.4s	remaining: 7.51s
672:	learn: 0.2260366	total: 15.4s	remaining: 7.49s
673:	learn: 0.2259222	total: 15.4s	remaining: 7.47s
674:	learn: 0.2257488	total: 15.5s	remaining: 7.45s
675:	learn: 0

821:	learn: 0.2095041	total: 18.8s	remaining: 4.08s
822:	learn: 0.2093937	total: 18.9s	remaining: 4.06s
823:	learn: 0.2092704	total: 18.9s	remaining: 4.03s
824:	learn: 0.2091877	total: 18.9s	remaining: 4.01s
825:	learn: 0.2090501	total: 18.9s	remaining: 3.99s
826:	learn: 0.2089466	total: 18.9s	remaining: 3.96s
827:	learn: 0.2088893	total: 19s	remaining: 3.94s
828:	learn: 0.2087865	total: 19s	remaining: 3.92s
829:	learn: 0.2087141	total: 19s	remaining: 3.9s
830:	learn: 0.2086083	total: 19s	remaining: 3.87s
831:	learn: 0.2085037	total: 19.1s	remaining: 3.85s
832:	learn: 0.2083884	total: 19.1s	remaining: 3.83s
833:	learn: 0.2082695	total: 19.1s	remaining: 3.8s
834:	learn: 0.2081788	total: 19.1s	remaining: 3.78s
835:	learn: 0.2080831	total: 19.2s	remaining: 3.76s
836:	learn: 0.2079973	total: 19.2s	remaining: 3.73s
837:	learn: 0.2079018	total: 19.2s	remaining: 3.71s
838:	learn: 0.2077813	total: 19.2s	remaining: 3.69s
839:	learn: 0.2077054	total: 19.2s	remaining: 3.67s
840:	learn: 0.2076056	

980:	learn: 0.1934675	total: 22.5s	remaining: 435ms
981:	learn: 0.1933708	total: 22.5s	remaining: 412ms
982:	learn: 0.1932980	total: 22.5s	remaining: 389ms
983:	learn: 0.1932256	total: 22.5s	remaining: 367ms
984:	learn: 0.1931625	total: 22.6s	remaining: 344ms
985:	learn: 0.1930385	total: 22.6s	remaining: 321ms
986:	learn: 0.1929546	total: 22.6s	remaining: 298ms
987:	learn: 0.1928721	total: 22.6s	remaining: 275ms
988:	learn: 0.1928205	total: 22.6s	remaining: 252ms
989:	learn: 0.1927416	total: 22.7s	remaining: 229ms
990:	learn: 0.1926858	total: 22.7s	remaining: 206ms
991:	learn: 0.1925936	total: 22.7s	remaining: 183ms
992:	learn: 0.1925069	total: 22.7s	remaining: 160ms
993:	learn: 0.1924555	total: 22.8s	remaining: 137ms
994:	learn: 0.1922318	total: 22.8s	remaining: 115ms
995:	learn: 0.1921670	total: 22.8s	remaining: 91.6ms
996:	learn: 0.1920453	total: 22.8s	remaining: 68.7ms
997:	learn: 0.1918891	total: 22.9s	remaining: 45.8ms
998:	learn: 0.1918162	total: 22.9s	remaining: 22.9ms
999:	lea

In [37]:
model = CatBoostClassifier(iterations=1000,
                           depth=10,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4454410	total: 88.4ms	remaining: 1m 28s
1:	learn: 0.4332463	total: 179ms	remaining: 1m 29s
2:	learn: 0.4272722	total: 267ms	remaining: 1m 28s
3:	learn: 0.4213994	total: 357ms	remaining: 1m 28s
4:	learn: 0.4134458	total: 447ms	remaining: 1m 28s
5:	learn: 0.4088030	total: 536ms	remaining: 1m 28s
6:	learn: 0.4037793	total: 622ms	remaining: 1m 28s
7:	learn: 0.4001495	total: 713ms	remaining: 1m 28s
8:	learn: 0.3967458	total: 799ms	remaining: 1m 27s
9:	learn: 0.3932589	total: 888ms	remaining: 1m 27s
10:	learn: 0.3897691	total: 972ms	remaining: 1m 27s
11:	learn: 0.3866226	total: 1.06s	remaining: 1m 27s
12:	learn: 0.3844184	total: 1.15s	remaining: 1m 27s
13:	learn: 0.3818279	total: 1.24s	remaining: 1m 27s
14:	learn: 0.3782521	total: 1.33s	remaining: 1m 27s
15:	learn: 0.3756656	total: 1.42s	remaining: 1m 27s
16:	learn: 0.3713357	total: 1.5s	remaining: 1m 26s
17:	learn: 0.3682312	total: 1.6s	remaining: 1m 27s
18:	learn: 0.3650763	total: 1.69s	remaining: 1m 27s
19:	learn: 0.3628092	to

158:	learn: 0.1664058	total: 14.6s	remaining: 1m 17s
159:	learn: 0.1654492	total: 14.7s	remaining: 1m 17s
160:	learn: 0.1647729	total: 14.8s	remaining: 1m 16s
161:	learn: 0.1642243	total: 14.9s	remaining: 1m 16s
162:	learn: 0.1632708	total: 15s	remaining: 1m 16s
163:	learn: 0.1623057	total: 15.1s	remaining: 1m 16s
164:	learn: 0.1613254	total: 15.1s	remaining: 1m 16s
165:	learn: 0.1606918	total: 15.2s	remaining: 1m 16s
166:	learn: 0.1596881	total: 15.3s	remaining: 1m 16s
167:	learn: 0.1590302	total: 15.4s	remaining: 1m 16s
168:	learn: 0.1582041	total: 15.5s	remaining: 1m 16s
169:	learn: 0.1571735	total: 15.6s	remaining: 1m 16s
170:	learn: 0.1564125	total: 15.7s	remaining: 1m 16s
171:	learn: 0.1557509	total: 15.8s	remaining: 1m 16s
172:	learn: 0.1553924	total: 15.9s	remaining: 1m 16s
173:	learn: 0.1549203	total: 16s	remaining: 1m 15s
174:	learn: 0.1542758	total: 16.1s	remaining: 1m 15s
175:	learn: 0.1531957	total: 16.2s	remaining: 1m 15s
176:	learn: 0.1523416	total: 16.3s	remaining: 1m 1

316:	learn: 0.0753586	total: 29.8s	remaining: 1m 4s
317:	learn: 0.0750644	total: 29.9s	remaining: 1m 4s
318:	learn: 0.0748205	total: 30s	remaining: 1m 3s
319:	learn: 0.0743569	total: 30.1s	remaining: 1m 3s
320:	learn: 0.0740220	total: 30.1s	remaining: 1m 3s
321:	learn: 0.0735023	total: 30.2s	remaining: 1m 3s
322:	learn: 0.0731845	total: 30.3s	remaining: 1m 3s
323:	learn: 0.0727973	total: 30.4s	remaining: 1m 3s
324:	learn: 0.0723907	total: 30.5s	remaining: 1m 3s
325:	learn: 0.0717643	total: 30.6s	remaining: 1m 3s
326:	learn: 0.0714857	total: 30.7s	remaining: 1m 3s
327:	learn: 0.0711512	total: 30.8s	remaining: 1m 3s
328:	learn: 0.0707976	total: 30.9s	remaining: 1m 3s
329:	learn: 0.0703849	total: 31s	remaining: 1m 2s
330:	learn: 0.0700247	total: 31.1s	remaining: 1m 2s
331:	learn: 0.0696480	total: 31.2s	remaining: 1m 2s
332:	learn: 0.0692635	total: 31.3s	remaining: 1m 2s
333:	learn: 0.0690672	total: 31.4s	remaining: 1m 2s
334:	learn: 0.0687347	total: 31.5s	remaining: 1m 2s
335:	learn: 0.06

477:	learn: 0.0349758	total: 44.8s	remaining: 48.9s
478:	learn: 0.0347792	total: 44.9s	remaining: 48.8s
479:	learn: 0.0345907	total: 45s	remaining: 48.7s
480:	learn: 0.0344491	total: 45.1s	remaining: 48.6s
481:	learn: 0.0342139	total: 45.2s	remaining: 48.5s
482:	learn: 0.0341133	total: 45.3s	remaining: 48.4s
483:	learn: 0.0339835	total: 45.3s	remaining: 48.3s
484:	learn: 0.0338220	total: 45.4s	remaining: 48.2s
485:	learn: 0.0337237	total: 45.5s	remaining: 48.2s
486:	learn: 0.0335833	total: 45.6s	remaining: 48.1s
487:	learn: 0.0334208	total: 45.7s	remaining: 48s
488:	learn: 0.0333146	total: 45.8s	remaining: 47.9s
489:	learn: 0.0331967	total: 45.9s	remaining: 47.8s
490:	learn: 0.0331119	total: 46s	remaining: 47.7s
491:	learn: 0.0329816	total: 46.1s	remaining: 47.6s
492:	learn: 0.0328497	total: 46.2s	remaining: 47.5s
493:	learn: 0.0326899	total: 46.3s	remaining: 47.4s
494:	learn: 0.0325524	total: 46.4s	remaining: 47.3s
495:	learn: 0.0324326	total: 46.4s	remaining: 47.2s
496:	learn: 0.0323

636:	learn: 0.0179156	total: 59.5s	remaining: 33.9s
637:	learn: 0.0178462	total: 59.6s	remaining: 33.8s
638:	learn: 0.0177774	total: 59.7s	remaining: 33.7s
639:	learn: 0.0176788	total: 59.8s	remaining: 33.6s
640:	learn: 0.0175845	total: 59.9s	remaining: 33.6s
641:	learn: 0.0175283	total: 60s	remaining: 33.5s
642:	learn: 0.0174501	total: 1m	remaining: 33.4s
643:	learn: 0.0174021	total: 1m	remaining: 33.3s
644:	learn: 0.0173316	total: 1m	remaining: 33.2s
645:	learn: 0.0172669	total: 1m	remaining: 33.1s
646:	learn: 0.0172090	total: 1m	remaining: 33s
647:	learn: 0.0171648	total: 1m	remaining: 32.9s
648:	learn: 0.0170822	total: 1m	remaining: 32.8s
649:	learn: 0.0170460	total: 1m	remaining: 32.7s
650:	learn: 0.0169633	total: 1m	remaining: 32.6s
651:	learn: 0.0168913	total: 1m	remaining: 32.5s
652:	learn: 0.0168252	total: 1m 1s	remaining: 32.4s
653:	learn: 0.0167792	total: 1m 1s	remaining: 32.3s
654:	learn: 0.0167072	total: 1m 1s	remaining: 32.2s
655:	learn: 0.0166527	total: 1m 1s	remaining: 

794:	learn: 0.0101642	total: 1m 14s	remaining: 19.2s
795:	learn: 0.0101284	total: 1m 14s	remaining: 19.1s
796:	learn: 0.0100954	total: 1m 14s	remaining: 19s
797:	learn: 0.0100527	total: 1m 14s	remaining: 18.9s
798:	learn: 0.0100226	total: 1m 14s	remaining: 18.8s
799:	learn: 0.0099877	total: 1m 14s	remaining: 18.7s
800:	learn: 0.0099589	total: 1m 14s	remaining: 18.6s
801:	learn: 0.0099336	total: 1m 14s	remaining: 18.5s
802:	learn: 0.0098944	total: 1m 15s	remaining: 18.4s
803:	learn: 0.0098779	total: 1m 15s	remaining: 18.3s
804:	learn: 0.0098545	total: 1m 15s	remaining: 18.2s
805:	learn: 0.0098129	total: 1m 15s	remaining: 18.1s
806:	learn: 0.0097941	total: 1m 15s	remaining: 18s
807:	learn: 0.0097644	total: 1m 15s	remaining: 17.9s
808:	learn: 0.0097427	total: 1m 15s	remaining: 17.9s
809:	learn: 0.0097215	total: 1m 15s	remaining: 17.8s
810:	learn: 0.0096967	total: 1m 15s	remaining: 17.7s
811:	learn: 0.0096532	total: 1m 15s	remaining: 17.6s
812:	learn: 0.0096112	total: 1m 15s	remaining: 17.

950:	learn: 0.0067293	total: 1m 28s	remaining: 4.58s
951:	learn: 0.0067247	total: 1m 29s	remaining: 4.49s
952:	learn: 0.0066888	total: 1m 29s	remaining: 4.39s
953:	learn: 0.0066785	total: 1m 29s	remaining: 4.3s
954:	learn: 0.0066571	total: 1m 29s	remaining: 4.21s
955:	learn: 0.0066570	total: 1m 29s	remaining: 4.11s
956:	learn: 0.0066380	total: 1m 29s	remaining: 4.02s
957:	learn: 0.0066220	total: 1m 29s	remaining: 3.93s
958:	learn: 0.0066026	total: 1m 29s	remaining: 3.83s
959:	learn: 0.0065771	total: 1m 29s	remaining: 3.74s
960:	learn: 0.0065620	total: 1m 29s	remaining: 3.65s
961:	learn: 0.0065619	total: 1m 29s	remaining: 3.55s
962:	learn: 0.0065547	total: 1m 30s	remaining: 3.46s
963:	learn: 0.0065359	total: 1m 30s	remaining: 3.37s
964:	learn: 0.0065117	total: 1m 30s	remaining: 3.27s
965:	learn: 0.0064905	total: 1m 30s	remaining: 3.18s
966:	learn: 0.0064892	total: 1m 30s	remaining: 3.08s
967:	learn: 0.0064711	total: 1m 30s	remaining: 2.99s
968:	learn: 0.0064490	total: 1m 30s	remaining: 

In [38]:
model = CatBoostClassifier(iterations=1000,
                           depth=10,
                           learning_rate=0.5,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.4944320	total: 88.2ms	remaining: 1m 28s
1:	learn: 0.4504171	total: 183ms	remaining: 1m 31s
2:	learn: 0.4377743	total: 272ms	remaining: 1m 30s
3:	learn: 0.4295708	total: 360ms	remaining: 1m 29s
4:	learn: 0.4237752	total: 453ms	remaining: 1m 30s
5:	learn: 0.4193015	total: 541ms	remaining: 1m 29s
6:	learn: 0.4152950	total: 627ms	remaining: 1m 28s
7:	learn: 0.4121041	total: 716ms	remaining: 1m 28s
8:	learn: 0.4088263	total: 806ms	remaining: 1m 28s
9:	learn: 0.4053573	total: 898ms	remaining: 1m 28s
10:	learn: 0.4032816	total: 988ms	remaining: 1m 28s
11:	learn: 0.4004234	total: 1.08s	remaining: 1m 28s
12:	learn: 0.3978869	total: 1.17s	remaining: 1m 28s
13:	learn: 0.3963962	total: 1.25s	remaining: 1m 28s
14:	learn: 0.3923782	total: 1.34s	remaining: 1m 28s
15:	learn: 0.3895993	total: 1.44s	remaining: 1m 28s
16:	learn: 0.3869857	total: 1.53s	remaining: 1m 28s
17:	learn: 0.3846312	total: 1.62s	remaining: 1m 28s
18:	learn: 0.3831488	total: 1.71s	remaining: 1m 28s
19:	learn: 0.3815590	

157:	learn: 0.2235017	total: 14.5s	remaining: 1m 17s
158:	learn: 0.2228245	total: 14.5s	remaining: 1m 16s
159:	learn: 0.2220425	total: 14.6s	remaining: 1m 16s
160:	learn: 0.2214214	total: 14.7s	remaining: 1m 16s
161:	learn: 0.2205935	total: 14.8s	remaining: 1m 16s
162:	learn: 0.2199760	total: 14.9s	remaining: 1m 16s
163:	learn: 0.2192025	total: 15s	remaining: 1m 16s
164:	learn: 0.2184396	total: 15.1s	remaining: 1m 16s
165:	learn: 0.2178039	total: 15.2s	remaining: 1m 16s
166:	learn: 0.2170636	total: 15.3s	remaining: 1m 16s
167:	learn: 0.2166332	total: 15.4s	remaining: 1m 16s
168:	learn: 0.2157429	total: 15.5s	remaining: 1m 16s
169:	learn: 0.2153336	total: 15.6s	remaining: 1m 16s
170:	learn: 0.2147372	total: 15.7s	remaining: 1m 16s
171:	learn: 0.2141640	total: 15.8s	remaining: 1m 15s
172:	learn: 0.2134317	total: 15.9s	remaining: 1m 15s
173:	learn: 0.2126659	total: 16s	remaining: 1m 15s
174:	learn: 0.2121010	total: 16s	remaining: 1m 15s
175:	learn: 0.2112985	total: 16.1s	remaining: 1m 15s

315:	learn: 0.1414863	total: 29.1s	remaining: 1m 2s
316:	learn: 0.1411975	total: 29.2s	remaining: 1m 2s
317:	learn: 0.1409041	total: 29.3s	remaining: 1m 2s
318:	learn: 0.1403074	total: 29.4s	remaining: 1m 2s
319:	learn: 0.1397945	total: 29.5s	remaining: 1m 2s
320:	learn: 0.1394853	total: 29.6s	remaining: 1m 2s
321:	learn: 0.1389919	total: 29.6s	remaining: 1m 2s
322:	learn: 0.1386491	total: 29.7s	remaining: 1m 2s
323:	learn: 0.1382940	total: 29.8s	remaining: 1m 2s
324:	learn: 0.1377615	total: 29.9s	remaining: 1m 2s
325:	learn: 0.1374155	total: 30s	remaining: 1m 2s
326:	learn: 0.1370332	total: 30.1s	remaining: 1m 1s
327:	learn: 0.1366362	total: 30.2s	remaining: 1m 1s
328:	learn: 0.1363517	total: 30.3s	remaining: 1m 1s
329:	learn: 0.1360648	total: 30.4s	remaining: 1m 1s
330:	learn: 0.1356901	total: 30.5s	remaining: 1m 1s
331:	learn: 0.1352291	total: 30.6s	remaining: 1m 1s
332:	learn: 0.1349229	total: 30.7s	remaining: 1m 1s
333:	learn: 0.1346726	total: 30.8s	remaining: 1m 1s
334:	learn: 0.

475:	learn: 0.0917452	total: 43.9s	remaining: 48.4s
476:	learn: 0.0914563	total: 44s	remaining: 48.3s
477:	learn: 0.0912541	total: 44.1s	remaining: 48.2s
478:	learn: 0.0909280	total: 44.2s	remaining: 48.1s
479:	learn: 0.0906407	total: 44.3s	remaining: 48s
480:	learn: 0.0904075	total: 44.4s	remaining: 47.9s
481:	learn: 0.0901604	total: 44.5s	remaining: 47.8s
482:	learn: 0.0899553	total: 44.6s	remaining: 47.7s
483:	learn: 0.0897570	total: 44.7s	remaining: 47.6s
484:	learn: 0.0894907	total: 44.8s	remaining: 47.6s
485:	learn: 0.0891775	total: 44.9s	remaining: 47.5s
486:	learn: 0.0889918	total: 45s	remaining: 47.4s
487:	learn: 0.0888344	total: 45.1s	remaining: 47.3s
488:	learn: 0.0885500	total: 45.2s	remaining: 47.2s
489:	learn: 0.0882813	total: 45.3s	remaining: 47.1s
490:	learn: 0.0880368	total: 45.3s	remaining: 47s
491:	learn: 0.0878588	total: 45.4s	remaining: 46.9s
492:	learn: 0.0876237	total: 45.5s	remaining: 46.8s
493:	learn: 0.0873817	total: 45.6s	remaining: 46.7s
494:	learn: 0.087226

634:	learn: 0.0614133	total: 58.8s	remaining: 33.8s
635:	learn: 0.0613206	total: 58.9s	remaining: 33.7s
636:	learn: 0.0610858	total: 59s	remaining: 33.6s
637:	learn: 0.0609433	total: 59.1s	remaining: 33.5s
638:	learn: 0.0607401	total: 59.2s	remaining: 33.4s
639:	learn: 0.0605670	total: 59.3s	remaining: 33.3s
640:	learn: 0.0603852	total: 59.4s	remaining: 33.3s
641:	learn: 0.0602646	total: 59.5s	remaining: 33.2s
642:	learn: 0.0601300	total: 59.5s	remaining: 33.1s
643:	learn: 0.0600225	total: 59.6s	remaining: 33s
644:	learn: 0.0598391	total: 59.7s	remaining: 32.9s
645:	learn: 0.0597260	total: 59.8s	remaining: 32.8s
646:	learn: 0.0596220	total: 59.9s	remaining: 32.7s
647:	learn: 0.0594442	total: 1m	remaining: 32.6s
648:	learn: 0.0593577	total: 1m	remaining: 32.5s
649:	learn: 0.0591952	total: 1m	remaining: 32.4s
650:	learn: 0.0590851	total: 1m	remaining: 32.3s
651:	learn: 0.0589847	total: 1m	remaining: 32.2s
652:	learn: 0.0588653	total: 1m	remaining: 32.1s
653:	learn: 0.0587376	total: 1m	re

794:	learn: 0.0423740	total: 1m 13s	remaining: 19s
795:	learn: 0.0422810	total: 1m 13s	remaining: 18.9s
796:	learn: 0.0421868	total: 1m 13s	remaining: 18.8s
797:	learn: 0.0420711	total: 1m 14s	remaining: 18.7s
798:	learn: 0.0419826	total: 1m 14s	remaining: 18.7s
799:	learn: 0.0418927	total: 1m 14s	remaining: 18.6s
800:	learn: 0.0417584	total: 1m 14s	remaining: 18.5s
801:	learn: 0.0416509	total: 1m 14s	remaining: 18.4s
802:	learn: 0.0415287	total: 1m 14s	remaining: 18.3s
803:	learn: 0.0414042	total: 1m 14s	remaining: 18.2s
804:	learn: 0.0413079	total: 1m 14s	remaining: 18.1s
805:	learn: 0.0412404	total: 1m 14s	remaining: 18s
806:	learn: 0.0411288	total: 1m 14s	remaining: 17.9s
807:	learn: 0.0410422	total: 1m 14s	remaining: 17.8s
808:	learn: 0.0409146	total: 1m 15s	remaining: 17.7s
809:	learn: 0.0408391	total: 1m 15s	remaining: 17.6s
810:	learn: 0.0407552	total: 1m 15s	remaining: 17.5s
811:	learn: 0.0406805	total: 1m 15s	remaining: 17.4s
812:	learn: 0.0406130	total: 1m 15s	remaining: 17.

951:	learn: 0.0302884	total: 1m 28s	remaining: 4.46s
952:	learn: 0.0302243	total: 1m 28s	remaining: 4.36s
953:	learn: 0.0301813	total: 1m 28s	remaining: 4.27s
954:	learn: 0.0301254	total: 1m 28s	remaining: 4.18s
955:	learn: 0.0300742	total: 1m 28s	remaining: 4.08s
956:	learn: 0.0300049	total: 1m 28s	remaining: 3.99s
957:	learn: 0.0299369	total: 1m 28s	remaining: 3.9s
958:	learn: 0.0298828	total: 1m 29s	remaining: 3.81s
959:	learn: 0.0298075	total: 1m 29s	remaining: 3.71s
960:	learn: 0.0297084	total: 1m 29s	remaining: 3.62s
961:	learn: 0.0296633	total: 1m 29s	remaining: 3.53s
962:	learn: 0.0295890	total: 1m 29s	remaining: 3.43s
963:	learn: 0.0295278	total: 1m 29s	remaining: 3.34s
964:	learn: 0.0294713	total: 1m 29s	remaining: 3.25s
965:	learn: 0.0294404	total: 1m 29s	remaining: 3.15s
966:	learn: 0.0293730	total: 1m 29s	remaining: 3.06s
967:	learn: 0.0293213	total: 1m 29s	remaining: 2.97s
968:	learn: 0.0292792	total: 1m 29s	remaining: 2.88s
969:	learn: 0.0292190	total: 1m 30s	remaining: 

In [39]:
model = CatBoostClassifier(iterations=1000,
                           depth=10,
                           learning_rate=0.3,
                           loss_function='Logloss',
                           verbose=True)
# train the model
model.fit(X_train,y_train)
# make the prediction using the resulting model
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
#print("class = ", preds_class)
#print("proba = ", preds_proba)
get_clf_eval(y_test, preds_class, preds_proba)

0:	learn: 0.5516256	total: 88.9ms	remaining: 1m 28s
1:	learn: 0.4910077	total: 181ms	remaining: 1m 30s
2:	learn: 0.4620933	total: 272ms	remaining: 1m 30s
3:	learn: 0.4480527	total: 364ms	remaining: 1m 30s
4:	learn: 0.4369096	total: 456ms	remaining: 1m 30s
5:	learn: 0.4306296	total: 550ms	remaining: 1m 31s
6:	learn: 0.4269239	total: 636ms	remaining: 1m 30s
7:	learn: 0.4230519	total: 733ms	remaining: 1m 30s
8:	learn: 0.4204729	total: 818ms	remaining: 1m 30s
9:	learn: 0.4167933	total: 905ms	remaining: 1m 29s
10:	learn: 0.4144465	total: 1.01s	remaining: 1m 30s
11:	learn: 0.4125669	total: 1.09s	remaining: 1m 30s
12:	learn: 0.4103789	total: 1.18s	remaining: 1m 29s
13:	learn: 0.4081421	total: 1.27s	remaining: 1m 29s
14:	learn: 0.4058494	total: 1.36s	remaining: 1m 29s
15:	learn: 0.4039277	total: 1.45s	remaining: 1m 29s
16:	learn: 0.4019844	total: 1.54s	remaining: 1m 29s
17:	learn: 0.3995875	total: 1.63s	remaining: 1m 28s
18:	learn: 0.3978720	total: 1.72s	remaining: 1m 28s
19:	learn: 0.3955034	

158:	learn: 0.2710386	total: 14.5s	remaining: 1m 16s
159:	learn: 0.2703262	total: 14.6s	remaining: 1m 16s
160:	learn: 0.2695254	total: 14.7s	remaining: 1m 16s
161:	learn: 0.2689501	total: 14.8s	remaining: 1m 16s
162:	learn: 0.2684985	total: 14.9s	remaining: 1m 16s
163:	learn: 0.2676720	total: 15s	remaining: 1m 16s
164:	learn: 0.2670937	total: 15.1s	remaining: 1m 16s
165:	learn: 0.2666874	total: 15.1s	remaining: 1m 16s
166:	learn: 0.2661683	total: 15.2s	remaining: 1m 15s
167:	learn: 0.2654161	total: 15.3s	remaining: 1m 15s
168:	learn: 0.2648363	total: 15.4s	remaining: 1m 15s
169:	learn: 0.2642210	total: 15.5s	remaining: 1m 15s
170:	learn: 0.2636925	total: 15.6s	remaining: 1m 15s
171:	learn: 0.2630803	total: 15.7s	remaining: 1m 15s
172:	learn: 0.2625550	total: 15.8s	remaining: 1m 15s
173:	learn: 0.2621490	total: 15.9s	remaining: 1m 15s
174:	learn: 0.2615495	total: 16s	remaining: 1m 15s
175:	learn: 0.2608335	total: 16.1s	remaining: 1m 15s
176:	learn: 0.2603883	total: 16.1s	remaining: 1m 1

315:	learn: 0.1984276	total: 28.9s	remaining: 1m 2s
316:	learn: 0.1980164	total: 29s	remaining: 1m 2s
317:	learn: 0.1976514	total: 29.1s	remaining: 1m 2s
318:	learn: 0.1972755	total: 29.2s	remaining: 1m 2s
319:	learn: 0.1968623	total: 29.3s	remaining: 1m 2s
320:	learn: 0.1964258	total: 29.4s	remaining: 1m 2s
321:	learn: 0.1959950	total: 29.5s	remaining: 1m 2s
322:	learn: 0.1955884	total: 29.6s	remaining: 1m 1s
323:	learn: 0.1951903	total: 29.7s	remaining: 1m 1s
324:	learn: 0.1948567	total: 29.7s	remaining: 1m 1s
325:	learn: 0.1946519	total: 29.8s	remaining: 1m 1s
326:	learn: 0.1942257	total: 29.9s	remaining: 1m 1s
327:	learn: 0.1938715	total: 30s	remaining: 1m 1s
328:	learn: 0.1934499	total: 30.1s	remaining: 1m 1s
329:	learn: 0.1930917	total: 30.2s	remaining: 1m 1s
330:	learn: 0.1929028	total: 30.3s	remaining: 1m 1s
331:	learn: 0.1926208	total: 30.4s	remaining: 1m 1s
332:	learn: 0.1922530	total: 30.5s	remaining: 1m 1s
333:	learn: 0.1918306	total: 30.6s	remaining: 1m
334:	learn: 0.19160

475:	learn: 0.1513756	total: 43.7s	remaining: 48.1s
476:	learn: 0.1510878	total: 43.8s	remaining: 48s
477:	learn: 0.1508714	total: 43.8s	remaining: 47.9s
478:	learn: 0.1506556	total: 43.9s	remaining: 47.8s
479:	learn: 0.1504361	total: 44s	remaining: 47.7s
480:	learn: 0.1502808	total: 44.1s	remaining: 47.6s
481:	learn: 0.1499170	total: 44.2s	remaining: 47.5s
482:	learn: 0.1496353	total: 44.3s	remaining: 47.4s
483:	learn: 0.1494033	total: 44.4s	remaining: 47.3s
484:	learn: 0.1492470	total: 44.5s	remaining: 47.2s
485:	learn: 0.1490139	total: 44.6s	remaining: 47.1s
486:	learn: 0.1488349	total: 44.7s	remaining: 47.1s
487:	learn: 0.1484547	total: 44.8s	remaining: 47s
488:	learn: 0.1481861	total: 44.9s	remaining: 46.9s
489:	learn: 0.1479519	total: 45s	remaining: 46.8s
490:	learn: 0.1476857	total: 45.1s	remaining: 46.7s
491:	learn: 0.1473777	total: 45.1s	remaining: 46.6s
492:	learn: 0.1471218	total: 45.2s	remaining: 46.5s
493:	learn: 0.1468180	total: 45.3s	remaining: 46.4s
494:	learn: 0.146680

634:	learn: 0.1161970	total: 58.5s	remaining: 33.7s
635:	learn: 0.1159782	total: 58.6s	remaining: 33.6s
636:	learn: 0.1157597	total: 58.7s	remaining: 33.5s
637:	learn: 0.1155690	total: 58.8s	remaining: 33.4s
638:	learn: 0.1153883	total: 58.9s	remaining: 33.3s
639:	learn: 0.1151913	total: 59s	remaining: 33.2s
640:	learn: 0.1150597	total: 59.1s	remaining: 33.1s
641:	learn: 0.1148682	total: 59.2s	remaining: 33s
642:	learn: 0.1146978	total: 59.3s	remaining: 32.9s
643:	learn: 0.1145324	total: 59.4s	remaining: 32.8s
644:	learn: 0.1143410	total: 59.5s	remaining: 32.8s
645:	learn: 0.1142036	total: 59.6s	remaining: 32.7s
646:	learn: 0.1140676	total: 59.7s	remaining: 32.6s
647:	learn: 0.1139021	total: 59.8s	remaining: 32.5s
648:	learn: 0.1136367	total: 59.9s	remaining: 32.4s
649:	learn: 0.1134817	total: 60s	remaining: 32.3s
650:	learn: 0.1132870	total: 1m	remaining: 32.2s
651:	learn: 0.1130464	total: 1m	remaining: 32.1s
652:	learn: 0.1128628	total: 1m	remaining: 32s
653:	learn: 0.1125662	total: 

795:	learn: 0.0901304	total: 1m 13s	remaining: 18.8s
796:	learn: 0.0899651	total: 1m 13s	remaining: 18.7s
797:	learn: 0.0898087	total: 1m 13s	remaining: 18.6s
798:	learn: 0.0896889	total: 1m 13s	remaining: 18.5s
799:	learn: 0.0895983	total: 1m 13s	remaining: 18.5s
800:	learn: 0.0894604	total: 1m 13s	remaining: 18.4s
801:	learn: 0.0893504	total: 1m 14s	remaining: 18.3s
802:	learn: 0.0892057	total: 1m 14s	remaining: 18.2s
803:	learn: 0.0891096	total: 1m 14s	remaining: 18.1s
804:	learn: 0.0889901	total: 1m 14s	remaining: 18s
805:	learn: 0.0888955	total: 1m 14s	remaining: 17.9s
806:	learn: 0.0887798	total: 1m 14s	remaining: 17.8s
807:	learn: 0.0886410	total: 1m 14s	remaining: 17.7s
808:	learn: 0.0885376	total: 1m 14s	remaining: 17.6s
809:	learn: 0.0884307	total: 1m 14s	remaining: 17.5s
810:	learn: 0.0882939	total: 1m 14s	remaining: 17.4s
811:	learn: 0.0881461	total: 1m 14s	remaining: 17.3s
812:	learn: 0.0880311	total: 1m 15s	remaining: 17.3s
813:	learn: 0.0879187	total: 1m 15s	remaining: 1

951:	learn: 0.0713439	total: 1m 27s	remaining: 4.43s
952:	learn: 0.0712413	total: 1m 28s	remaining: 4.34s
953:	learn: 0.0710919	total: 1m 28s	remaining: 4.25s
954:	learn: 0.0709998	total: 1m 28s	remaining: 4.16s
955:	learn: 0.0708760	total: 1m 28s	remaining: 4.06s
956:	learn: 0.0707222	total: 1m 28s	remaining: 3.97s
957:	learn: 0.0705558	total: 1m 28s	remaining: 3.88s
958:	learn: 0.0704366	total: 1m 28s	remaining: 3.79s
959:	learn: 0.0703025	total: 1m 28s	remaining: 3.69s
960:	learn: 0.0702173	total: 1m 28s	remaining: 3.6s
961:	learn: 0.0701453	total: 1m 28s	remaining: 3.51s
962:	learn: 0.0700971	total: 1m 28s	remaining: 3.42s
963:	learn: 0.0700130	total: 1m 29s	remaining: 3.33s
964:	learn: 0.0699140	total: 1m 29s	remaining: 3.23s
965:	learn: 0.0698180	total: 1m 29s	remaining: 3.14s
966:	learn: 0.0697458	total: 1m 29s	remaining: 3.05s
967:	learn: 0.0696396	total: 1m 29s	remaining: 2.96s
968:	learn: 0.0695434	total: 1m 29s	remaining: 2.86s
969:	learn: 0.0694207	total: 1m 29s	remaining: 